<a href="https://colab.research.google.com/github/parkerburchett/EconCapstone/blob/main/Ping%20and%20Group%20all%20Miner%20Income.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Constants
parker_wallet = '0x76fb6d38f28c44a13380220df21363bd7af45ee1'
ethan_wallet = '0xceb4d0ca821420cf2553b9e244f6b52364613f94'

## This is the first draft of the notebook I am writing for my Economics Capstone to get a published paper. \

https://www.etherchain.org/charts Look at "mining revenue" gives USD value per day of a GH/s. Use this instead of building it yourself.
### Outline: What will this notebook do?

1. Use the Etherscan.io charts to convert into production 
* Block Number to Date.(Year, Month)
* (Year, Month) to Monthly average Value of 1 day of mining at 1 GH/s (Etherscan.io chart). Just average by month. 

* (Year, Month) to Average ETH Price. 


* You will need a dictionary that looks like 
(Year, Month): Average Gh/s needed to mine make 1 ETH 
### Called production dictionary. 

2. You will need to write custom Estimate_hashrate 

*Pseudocode:
  Estimate_HashRate(Year, Month, ETH earned this month):

  estimated_hashrate_in_ghS = Production_Dictionary[(Year, Month) * ETH earned  this month)


3. Use the Etherscan.io API to get all of the transactions from 5 large pools, and store that in pyspark on Google Colab. You will need to save that to someplace in the cloud so that you can get it later and put the public link. I think drop box is the right place for it.

 -> What I want the data to look like.
 This information is factual raw data. 
(Block Number of Transaction, Pool Address, Miner Address, Amount of ETH)


Every Single Transaction will be mapped into this form. 

(Block Number, Day, Year, Month, Pool Address, Miner Address, Amount of ETH, Estimated_hashrate)

This data will then be grouped by Miner_address and Month.

Intermediate(after group by) result:

Month, Year, Pool Address, Miner Address, ETH Earned this month, Estimated Hashrate, (ETH earned this month * Average Monthly Price of ETH) as monthly USD Revenue.

Frame work for Miner Features
This is how I am choosing to categorize Miners. 

## Miner Features
Start month ( cast this as an int with January 2015 as 1). # month of first  income).
End Month (cast this as an int with March 2021 as the 80 or whatever the number happens to be). Last month with income. or + infinity if that was march 2021. Age as number of full months,
Median Hashrate, as size. 



Dummy Varibles for Pool address, 
Dummy Variables for if they went to a different




Framework for OLS regression:

##Independent variable: 
* Estimated Monthly Hashrate.

###Dependent Variables
Miner Features. 
Year
Month,
Average ETH price,
average Gh/s value



### Setup Pyspark and Connect it to your google drive

In [2]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

# Open Connection to Google drive

from google.colab import drive
drive.mount('/content/drive')
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
print("\n\nDRIVE SETUP")

openjdk-8-jdk-headless is already the newest version (8u282-b08-0ubuntu1~18.04).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


 DRIVE SETUP


## Install Libraries 

In [3]:
import pandas as pd
import numpy as np
import json
import requests
import math
import datetime
import matplotlib.pyplot as plt
import time
%matplotlib inline

import pyspark
from functools import reduce
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import DataType
from pyspark import SparkContext, SparkConf


### Create Spark Session

In [4]:
# create the spark session
conf = SparkConf().set("spark.ui.port", "4050") # copy and pasted this, dont' know what it means
# create the context
# try catch block lets you start again from the top without causing problems. 
try:
  sc.stop()
  sc = pyspark.SparkContext(conf=conf)
except:
  sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

print('Successfully Created Spark Session')

Successfully Created Spark Session


## Establish constants

In [5]:
def read_api_key():
  api_key_json = open(r'/content/drive/MyDrive/etherScan_apiKey.json') # read in my private Etherscan.io API key from my Google Drive
  etherscan_api_key =json.load(api_key_json)['key']
  api_key_json.close()
  return etherscan_api_key

ETHERSCAN_API_KEY = read_api_key() # You api key is a constant so the variable name is UPPER CASE
ethermine_wallet = '0xea674fdde714fd979de3edf0f56aa9716b898ec8'
def query_normal_transactions(wallet_address, startblock=0, endblock=99999999): 
  """
    Ping Etherscan.io and get the most recent 10k transactions for this wallet.
    Each of these takes ~4 seconds
  """
  api_request_text =f'https://api.etherscan.io/api?module=account&action=txlist&address={wallet_address}&startblock={startblock}&endblock={endblock}&sort=asc&apikey={ETHERSCAN_API_KEY}'
     
  response = requests.get(api_request_text) # Need to throttle this to 5 calls a second
  api_response_text = json.loads(response.text) 
  # error handling to make sure that the call is going through
  if api_response_text['message'] != 'OK': # untested
    print(f'you got an error at address={wallet_address} \nstartblock= {startblock}\nendblock={endblock}')

  else:
    transaction_list = api_response_text['result']
    simplified_transactions = parse_normal_transactions(transaction_list=transaction_list, wallet_address=wallet_address)

    #exclude the last N transactions since the might overlapp
    largest_block = int(simplified_transactions[-1][2]) # O(1)
    while simplified_transactions[-1][2] == largest_block:
      simplified_transactions.pop(-1) # O(1) * 100 upper bound.

    column_names = ['to_address',
                    'from_address',
                    'block_number',
                    'transaction_year',
                    'transaction_month',
                    'value_in_ether']
    
    # create a rdd of this batch of transactions
    rdd = spark.createDataFrame(simplified_transactions, schema=column_names)
    return rdd, largest_block

def parse_normal_transactions(transaction_list: list, wallet_address):
    """
      Converts transaction data into a easy to read from and excludes the last block
    :param transaction_list: A list of dictionary objects. Each dictionary is a single transaction.
    :return:
        simple_transactions: a list of tuples storing (to_address, from_address, Block number, Value in Ether)
                                      Data types are: (String, String, int, year,, float)
    """
    simplified_transactions =[(trans_dict['to'],
                              trans_dict['from'],
                              int(trans_dict['blockNumber']),
                              datetime.datetime.fromtimestamp(int(trans_dict['timeStamp'])).year,
                               datetime.datetime.fromtimestamp(int(trans_dict['timeStamp'])).month,
                              int(trans_dict['value'])/math.pow(10, 18))
                              for trans_dict in transaction_list if trans_dict['from'] == wallet_address
                              ]
    return simplified_transactions

## Next steps are to get a dictionary of block: time chunks. 




In [6]:
start_year = 2016
years = [datetime.datetime(start_year+i,1,1) for i in range(0,6)]
unix_timestamp_years = [int(year.timestamp()) for year in years]
unix_timestamp_march31_2016 = int(datetime.datetime(2021,3,31).timestamp())

command_for_block_after_timestamp = f'https://api.etherscan.io/api?module=block&action=getblocknobytime&timestamp={unix_timestamp_years[0]}&closest=after&apikey={ETHERSCAN_API_KEY}'

year_block_dict ={}

for unix_time in unix_timestamp_years:
  time.sleep(.2) # don't overwhelm the API
  response = requests.get(f'https://api.etherscan.io/api?module=block&action=getblocknobytime&timestamp={unix_time}&closest=after&apikey={ETHERSCAN_API_KEY}')
  response_text = json.loads(response.text) 
  block = response_text['result']
  year_block_dict[int(datetime.datetime.fromtimestamp(unix_time).year)]= block

print(year_block_dict)
response = requests.get(f'https://api.etherscan.io/api?module=block&action=getblocknobytime&timestamp={unix_timestamp_march31_2016}&closest=after&apikey={ETHERSCAN_API_KEY}')
response_text = json.loads(response.text) 
march_2021_block = response_text['result']
march_2021_block
year_block_dict['2022'] = march_2021_block # this is to make the code work. as written 

{2016: '778483', 2017: '2912407', 2018: '4832686', 2019: '6988615', 2020: '9193266', 2021: '11565019'}


In [7]:
def get_monthly_miner_income_statements_chunk(year,pool_address):
  """
    Ping every transaction where {pool_address} is the sender and cast it as a pyspark dataframe.

    add a year_month column. 

    Group by to_address and year_month

    cast as a pandas df

    Save the document to a file in this local runtime of google drive. 
  """
  start = datetime.datetime.now()
  min_block = int(year_block_dict[year]) # if year =2018 then this gets all miner incomes in 2018
  max_block = int(year_block_dict[year+1]) # all blocks must be within this range
  i=0
  transactions_rdd, largest_block = query_normal_transactions(wallet_address=pool_address, startblock=min_block, endblock=max_block)

  while largest_block < max_block:
    try:
      # repeat the call picking up where you left off last call since you can only get 10,000 records per call
      next_chunk_rdd, largest_block = query_normal_transactions(wallet_address=pool_address, startblock=largest_block) 
      transactions_rdd = transactions_rdd.union(next_chunk_rdd) # union removes duplicates
      time_dif = datetime.datetime.now() - start
      start = datetime.datetime.now()
      # this lets you see progress
      print(f'Call:{i} Time:{time_dif} Block:{largest_block} Blocks Left:{max_block - largest_block}')
      i+=1
    except:
      print(f'you got an error at {largest_block}\n {next_chunk_rdd.head(1)} ')

  
  # add the year_month column. 
  transactions_rdd = transactions_rdd.withColumn('year_month', concat(transactions_rdd.transaction_year,lit("-"),
                                                                      transactions_rdd.transaction_month))

  transactions_rdd.registerTempTable("raw_transactions") # I am pretty sure I don't need to deregister this. Unverified


  monthly_miner_revenue_query =f""" SELECT to_address, year_month, ROUND(sum(value_in_ether),9) as eth_earned
                                    FROM raw_transactions 
                                    Where block_number < {max_block} and block_number > {min_block}
                                    GROUP BY to_address, year_month
                                    ORDER BY to_address, year_month 
                                """

  miners_sql_df = spark.sql(monthly_miner_revenue_query)
  miners_sql_df = miners_sql_df.withColumn('from_address', lit(pool_address)) # add the pool address back into the rdd

  df = miners_sql_df.toPandas()
  print('Verify that these months are correct')
  print(df['year_month'].unique())
  df.to_csv(f'/content/drive/MyDrive/transaction_data/miner_income_{year}_{pool_address}.csv',index=False) # save to your google drive


In [8]:
%%time

# first test
# get_monthly_miner_income_statements_chunk(2016,ethermine_wallet)

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 8.82 µs


In [9]:
HIVEON_POOL= '0x1aD91ee08f21bE3dE0BA2ba6918E714dA6B45836'
SPARK_POOL = '0x5A0b54D5dc17e0AadC383d2db43B0a0D3E029c4c'
F2_POOL = '0x829BD824B016326A401d083B33D092293333A830'
ZHIZHU_TOP_POOL = '0x04668Ec2f57cC15c381b461B9fEDaB5D451c8F7F'
ETHERMINE_POOL = '0xEA674fdDe714fd979de3EdF0F56AA9716B898ec8'

POOLS = [HIVEON_POOL, SPARK_POOL, F2_POOL, ZHIZHU_TOP_POOL, ETHERMINE_POOL]

In [ ]:
%%time
years = [2016,2017,2018,2019,2020,2021]
for y in years:
  get_monthly_miner_income_statements_chunk(y,ethermine_wallet)



Call:0 Time:0:00:14.548862 Block:1312289 Blocks Left:1600118
Call:1 Time:0:00:04.069931 Block:1347595 Blocks Left:1564812
Call:2 Time:0:00:05.801691 Block:1378032 Blocks Left:1534375
Call:3 Time:0:00:03.584586 Block:1407645 Blocks Left:1504762
Call:4 Time:0:00:04.458185 Block:1435260 Blocks Left:1477147
Call:5 Time:0:00:04.125148 Block:1460803 Blocks Left:1451604
Call:6 Time:0:00:04.368528 Block:1487553 Blocks Left:1424854
Call:7 Time:0:00:03.859156 Block:1512372 Blocks Left:1400035
Call:8 Time:0:00:03.782743 Block:1536853 Blocks Left:1375554
Call:9 Time:0:00:04.036393 Block:1557198 Blocks Left:1355209
Call:10 Time:0:00:03.818006 Block:1579298 Blocks Left:1333109


## Methods to get transactions from Etherscan.io API


### You will want some try catch logic to make sure that you are not needlessly pinging Etherscan.io

### Figure out what is the first and last block in each year. 




In [ ]:
# %%time
# start = datetime.datetime.now()
# ethermine_transaction_rdd, largest_block = query_normal_transactions(wallet_address=ethermine_wallet)
# for i in range(4000): # the large call should be 4,000 ish. Just let it run while you are making dinner. see what happens.  
#   try:
#     next_chunk_rdd, largest_block = query_normal_transactions(wallet_address=ethermine_wallet, startblock=largest_block)
#     ethermine_transaction_rdd = ethermine_transaction_rdd.union(next_chunk_rdd)
#     if largest_block >= 12143978: # this the last block you want to consider
#       break
#     time_dif = datetime.datetime.now() -start
#     start = datetime.datetime.now() 
#     print(f'Call:{i+1} Time:{time_dif}')
#   except:
#     print(f'you got an error at {largest_block}\n {next_chunk_rdd.head(1)} ')


### Only save the group by month_data



In [ ]:
# rdd = ethermine_transaction_rdd
# rdd = rdd.withColumn('year_month', concat(rdd.transaction_year,lit("-"),rdd.transaction_month)) # add the year_month column

In [ ]:
# rdd.registerTempTable("ethermine_transactions")

# monthly_miner_revenue_query ="""SELECT to_address, year_month, ROUND(sum(value_in_ether),9) as eth_earned
# FROM ethermine_transactions 
# GROUP BY to_address, year_month
# ORDER BY to_address, year_month 
# """

# miners = spark.sql(monthly_miner_revenue_query)
# num_monthly_income_statements = miners.count()
# print(f'You have {num_monthly_income_statements} income statements')
# miners = miners.withColumn('from_address', lit(ethermine_wallet))
# miners.columns

## Save miner_grouped_rdd to a .csv file on your google drive. 



In [ ]:
# miners_df = miners.toPandas()
# miners_df.to_csv(r'/content/drive/MyDrive/miners.csv', index=False) # unclear what happens when there is alreadya file with stuff here.


In [ ]:
# with open(r'/content/drive/MyDrive/miners.csv', 'r') as data:
#   num_lines_in_file = len(data.readlines())
#   print(f'In your google drive you saved a file with: {num_lines_in_file} lines')
#   print(f'There should be:                            {num_monthly_income_statements+1} lines ')

### Estimated Timecosts 

In [ ]:
# # How many API calls do you need to make?
# n_transactions = 40000000
# reccords_per_call = 10000
# total_api_calls = n_transactions/reccords_per_call
# base_time_cost_per_api_call = 5 #Loose Uppperbound
# seconds_needed = (base_time_cost_per_api_call *total_api_calls) 
# hours = seconds_needed/3600
# print(f'Upper bound for API calls for ethermine {total_api_calls_needed}') # upper bound is 5 hours.
# print(f'Upper bound for hours to get all of Ethermine hours to get everything from ethermine {hours}')